In [ ]:
import cv2
import numpy as np
import math

video = cv2.VideoCapture("test.avi")
history = []
    
while True:
    ret, frame = video.read()
    if not ret:
        video = cv2.VideoCapture("test.avi")
        continue
    
    # take R out of BGR
    b,g,r = cv2.split(frame)
    
    # gaussian blur
    blurred = cv2.GaussianBlur(r,(5,5),0)
    
    # canny
    Threshold1 = 15;
    Threshold2 = 200;
    FilterSize = 1
    E = cv2.Canny(blurred, Threshold1, Threshold2, FilterSize)

    # lines
    # Rres = 1
    # Thetares = 1*np.pi/180
    # Threshold = 100
    # minLineLength = 10
    # maxLineGap = 100
    # cv2.HoughLines(edges, 1, np.pi / 180, 190)
    # (edges, lines, 1, CV_PI / 180, 100, 100, 10);
    # lines = cv2.HoughLinesP(E,3, np.pi/180, 100, 100, 10)
    max_slider = 100
    lines = cv2.HoughLinesP(E, 1, np.pi/180, max_slider, minLineLength=50, maxLineGap=50)

    max_x = 640
    max_y = 480
    
    if lines is not None:
        N = lines.shape[0]
        serious_lines = []
        
        left_xs = []
        right_xs = []

        for i in range(N):
            x1, y1, x2, y2 = lines[i][0]

            dx = x2-x1
            dy = y2-y1
            angle = np.arctan2(dy,dx) * (180/np.pi)
            length = np.sqrt(dx**2+dy**2)

            delta = 60
            if abs(angle) > 90-delta and abs(angle) < 90+delta:
                color = (0,0,255)
                
                dx = abs(x2)-abs(x1)
                dy = abs(y2)-abs(y1)
                dxdy = abs(dx/dy)

                y3 = 0 
                
                if angle < 0:
                    # left line
                    color = (0, 255, 0)
                    x3 = x2 + int(dxdy*y2)
                    left_xs.append(x3)
                else:
                    # right line
                    x3 = x2 - int(dxdy*y2)
                    right_xs.append(x3)
                        
                cv2.line(frame,(x2,y2),(x3,y3),(255, 0, 0),2)
                
                cv2.line(frame,(x1,y1),(x2,y2),color,2)
                
                print("Original end point", x2, y2)
                print("Start point", x1, y1)
                print("Angle", angle)
                print("Length", length)

                serious_lines.append((x1, y1, angle, length))

        sorted_lines = sorted(serious_lines, key=lambda tup: tup[2])
    #     print(sorted_lines)

        actual_lines = []
        

        
        
        def avg(val):
            if len(val) > 0:
                return np.average(val)
        
        def write(txt, lineno=1):
            cv2.putText(frame, txt, (10,340+40*lineno), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (209, 80, 0, 255), 3)
        
        x_left = avg(left_xs)
        x_right = avg(right_xs)
        
        print("History", history[-3:])
        
        left_history_avg = avg([x.get('x_left') for x in history[-3:] if x.get('x_left') is not None])
        right_history_avg = avg([x.get('x_right') for x in history[-3:] if x.get('x_right') is not None])
        
        guess = int(avg([x for x in [x_left, x_right, left_history_avg, right_history_avg] if x is not None]))
        cv2.line(frame,(guess,0),(guess,480),(255, 0, 0),2)
        
        if guess
        
        print("History left, right", left_history_avg, right_history_avg)
        
        moving_delta = []
        
        if x_left is not None and x_right is not None:
            space_left = x_left
            space_right = 640-x_right
            space_diff = space_left-space_right
#             write(f'Diff {space_diff} ({space_left}, {space_right})')  
            
#             if abs(space_diff) > 35:
#                 if space_diff > 0:
#                     write('Steer right', 2)
#                 else:
#                     write('Steer left', 2)
                
            text = "Moving "
            if left_history_avg is not None:
                delta = int(x_left-left_history_avg)
                moving_delta.append(delta)
                text += f"left {delta};"
            
            if right_history_avg is not None:
                delta = int(x_right-right_history_avg)
                moving_delta.append(delta)
                text += f"right {delta};"
            
            write(text, 3)
                
        elif x_left is not None:
            space_left = int(x_left)
            space_diff = 320-space_left
#             write(f'Space left = {space_left} / diff {space_left}')
            
            #if abs(space_diff) > 35:
            #    if space_diff > 0:
            #        write('Steer right', 2)
            #    else:
            #       write('Steer left', 2)
            
            text = "Moving "
            if left_history_avg is not None:
                delta = int(x_left-left_history_avg)
                moving_delta.append(delta)
                text += f"left {delta};"
            
            write(text, 3)
        elif x_right is not None:
            space_right = int(640-x_right)
            space_diff = 320-space_right
#             write(f'Space right = {space_right} / diff {space_diff}')
            
#             if abs(space_diff) > 35:
#                 if space_diff > 0:
#                     write('Steer right', 2)
#                 else:
#                     write('Steer left', 2)
            
            text = "Moving "
            if right_history_avg is not None:
                delta = int(x_right-right_history_avg)
                text += f"right {delta};"
            
            write(text, 3)
        else:
            write('No data')
        
        action = "do nothing"
        avg_moving_delta = avg(moving_delta)
        if avg_moving_delta is not None and abs(avg_moving_delta) > 10:
            if avg_moving_delta > 0:
                action = "steer right"
            else:
                action = "steer left"

        write(f"Mov.del. {avg(moving_delta)}; {action}", 2)
            
        history.append({
            "x_left": x_left,
            "x_right": x_right
        }) 

        prev = 0
        for x1,y2,angle,length in sorted_lines:
            delta = np.abs(np.abs(prev)-np.abs(angle))
            if delta > 1:
                actual_lines.append([[x1, x2, angle, length]])
            else:
                actual_lines[-1].append([x1, x2, angle, length])
            prev = angle

        print("Final lines", actual_lines)
        #print("Final lines", list(zip(actual_lines[1])))
    else:
        print("No lines")

    cv2.imshow("frame", frame)
    cv2.imshow("edges", E)
    
    key = cv2.waitKey(1025)
    if key == 27:
        break
        

cv2.destroyAllWindows()
video.release()

In [ ]:
def avg(val):
    if len(val) > 0: return np.average(val)

a = []
a.append(1)
print(avg(a))